In [ ]:
import pandas as pd
import geopandas as gpd
import glob
import os
import matplotlib.pyplot as plt

In [ ]:
# hours
path = r"C:\Users\ruome\Desktop\New folder\Renewable\ProcessingFiles\table\电网排放因子和成本-碳价参数0110V3.xlsx"
df_hours=pd.read_excel(path, sheet_name="2022年分省风光小时数")
df_hours = df_hours.iloc[4:,-4:]
df_hours.columns = ['Shengcode', '省份', '风', '光']
df_hours.fillna(0, inplace=True)
df_hours = df_hours.astype({'Shengcode':int, '风': float, '光': float})

df_parameter_summary = pd.read_excel(path, sheet_name='成本参数及碳价参数')
col = pd.read_excel(path, sheet_name="Columns")
df_parameter_summary.columns = col.iloc[:,1].to_list()
df_parameter_summary = df_parameter_summary.iloc[2:]

df_shengcode = pd.read_excel(path, sheet_name="Shengcode")

算装机容量总成本：Cost
1. 政策情景网格成本=网格装机容量 * 所在省份的单位装机成本（单位瓦价格）；
2. 高速情景网格成本=网格装机容量 * 所在省份的单位装机成本（单位瓦价格）；

如果省份没有单位装机成本值，用所在区域平均值计算。


计算减排量：Emission
1. 政策情景网格减排量=网格装机容量 * 所在省份的发电小时数 * 电网排放因子；
2. 高速情景网格减排量=网格装机容量 * 所在省份的发电小时数 * 电网排放因子；

如果省份没有电网排放因子，用sheet“电网排放因子”中的数据，这个省份是最全的。其中“低值”代表的是“高速情景”；“高值”代表的是“政策情景”。


计算利润：Profile
1. 政策情景网格利润=网格减排量*碳价-网格成本；
2. 高速情景网格利润=网格减排量*碳价-网格成本。

In [ ]:
def find_str(search_list, olist):
    result = [col for col in olist if all(term in col for term in search_list)]
    return result[0]
def cal2(df, df_hours, shp_path, sel_str, scenario):
    # Filter data
    df_value_sel = df.loc[df['风/光'].str.contains(sel_str, case=False, na=False)].copy()
    # Convert columns to numeric
    cols = df_value_sel.columns[12:]
    df_value_sel[cols] = df_value_sel[cols].apply(pd.to_numeric, errors='coerce')
    # Calculate mean values by province
    df_value_sel_mean = df_value_sel.groupby('Shengcode')[cols].mean().reset_index()
    df_value_sel_mean_daqu = df_value_sel.groupby('大区')[cols].mean()
    df_value_sel_mean = df_value_sel_mean.merge(df_shengcode.loc[:,['Shengcode','大区']], how = 'right', on = 'Shengcode')

    # Fill missing values by region
    for col in cols:
        for region in df_value_sel_mean_daqu.index:
            mean_value = df_value_sel_mean_daqu.loc[region, col]
            df_value_sel_mean.loc[df_value_sel_mean['大区'] == region, col] = df_value_sel_mean.loc[df_value_sel_mean['大区'] == region, col].fillna(mean_value)
    
    # Load shapefile data
    shp_df = gpd.read_file(shp_path)
    shp_crs = shp_df.crs
    shp_df['Shengcode'] = shp_df['Shengcode'].astype('int')
    shp_df = shp_df.merge(df_value_sel_mean, how='left', on='Shengcode')
    shp_df = shp_df.merge(df_hours.loc[:,['Shengcode',sel_str]], how = 'left', on = 'Shengcode')
    
    # Calculate cost, emission, and profit for 2022
    olist = df.columns
    shp_df['C2022'] = shp_df['KW2022'] * shp_df[find_str(['2022','单位瓦价格'], olist)] * 1000
    shp_df['E2022'] = shp_df['KW2022'] * shp_df[find_str(['2022','电力排放因子'],olist)] * shp_df[sel_str]
    shp_df['P2022'] = (shp_df['E2022']/1000) * shp_df[find_str(['2022','碳价'], olist)] - shp_df['C2022']

    # Calculate cost, emission, and profit for future years
    future_years = ['2025', '2030', '2035', '2040', '2050', '2060']
    for year in future_years:
        shp_df[f'C{year}'] = shp_df[f'KW{year}'] * shp_df[find_str([scenario, year,'单位瓦价格'],olist)] * 1000 
        shp_df[f'E{year}'] = shp_df[f'KW{year}'] * shp_df[sel_str] * shp_df[find_str([scenario, year,'电网因子'],olist)]
        shp_df[f'P{year}'] = (shp_df[f'E{year}']/1000) * shp_df[find_str([scenario, year,'碳价'],olist)] - shp_df[f'C{year}']
    
    return shp_df.loc[:,['NID10', 'Shengcode', 'NID50', 'NID10a', 'KW2',
                         'KW2022', 'KW2025', 'KW2030', 'KW2035', 'KW2040', 'KW2050','KW2060', 
                         'C2022','C2025','C2030','C2035','C2040','C2050','C2060',
                         'E2022','E2025','E2030','E2035','E2040','E2050','E2060',
                         'P2022','P2025','P2030','P2035','P2040','P2050','P2060',
                         'geometry']].set_crs(shp_crs)


def cal3(df, sel_str):
    # Filter data
    df_value_sel = df.loc[df['风/光'].str.contains(sel_str, case=False, na=False)].copy()
    # Convert columns to numeric
    cols = df_value_sel.columns[12:]
    df_value_sel[cols] = df_value_sel[cols].apply(pd.to_numeric, errors='coerce')
    # Calculate mean values by province
    df_value_sel_mean = df_value_sel.groupby('Shengcode')[cols].mean().reset_index()
    df_value_sel_mean_daqu = df_value_sel.groupby('大区')[cols].mean()
    df_value_sel_mean = df_value_sel_mean.merge(df_shengcode.loc[:,['Shengcode','大区']], how = 'right', on = 'Shengcode')

    # Fill missing values by region
    for col in cols:
        for region in df_value_sel_mean_daqu.index:
            mean_value = df_value_sel_mean_daqu.loc[region, col]
            df_value_sel_mean.loc[df_value_sel_mean['大区'] == region, col] = df_value_sel_mean.loc[df_value_sel_mean['大区'] == region, col].fillna(mean_value)
    return df_value_sel_mean

In [ ]:
sel_str = "风"
df = cal3(df_parameter_summary, sel_str)
df = df.to_excel(r"C:\Users\ruome\Desktop\Renewable\ProcessingFiles\table\feng.xlsx")

In [ ]:
sel_str = "光"
df = cal3(df_parameter_summary, sel_str)
df = df.to_excel(r"C:\Users\ruome\Desktop\Renewable\ProcessingFiles\table\guang.xlsx")

In [ ]:
shp_path = r"C:\Users\ruome\Desktop\New folder\Renewable\论文编辑\数据输出\结果二\wind_high_20240320.shp"
sel_str = "风"
scenario = "高速"
shp_df = cal2(df_parameter_summary, df_hours, shp_path, sel_str, scenario)
shp_df.to_file(r"C:\Users\ruome\Desktop\New folder\Renewable\论文编辑\数据输出\结果三-全生命周期折减\wind_high_results_20250110gj.shp")

In [ ]:
shp_path = r"C:\Users\ruome\Desktop\New folder\Renewable\论文编辑\数据输出\结果二\wind_median_20240320.shp"
sel_str = "风"
scenario = "政策"
shp_df = cal2(df_parameter_summary, df_hours, shp_path, sel_str, scenario)
shp_df.to_file(r"C:\Users\ruome\Desktop\New folder\Renewable\论文编辑\数据输出\结果三-全生命周期折减\wind_median_results_20250110gj.shp")

In [ ]:
shp_path = r"C:\Users\ruome\Desktop\New folder\Renewable\论文编辑\数据输出\结果二\Solar_median_20240226.shp"
sel_str = "光"
scenario = "政策"
shp_df = cal2(df_parameter_summary, df_hours, shp_path, sel_str, scenario)
shp_df.to_file(r"C:\Users\ruome\Desktop\New folder\Renewable\论文编辑\数据输出\结果三-全生命周期折减\Solar_median_results_20250110gj.shp")

In [ ]:
shp_path = r"C:\Users\ruome\Desktop\New folder\Renewable\论文编辑\数据输出\结果二\Solar_high_20240226.shp"
sel_str = "光"
scenario = "高速"
shp_df = cal2(df_parameter_summary, df_hours, shp_path, sel_str, scenario)
shp_df.to_file(r"C:\Users\ruome\Desktop\New folder\Renewable\论文编辑\数据输出\结果三-全生命周期折减\Solar_high_results_20250110gj.shp")

占比能达到多大的量级 

占比 ： 风光装机总量，和政策下风光的装机，和高情景下风光的装机对比

预想条件20% - 40%

小于0的选大，大于0的加和；现状装机量保留

In [ ]:
# Summary table
import glob
import geopandas as gpd
import pandas as pd
path_lst = glob.glob(r"C:\Users\ruome\Desktop\New folder\Renewable\论文编辑\数据输出\结果三-全生命周期折减\*.shp")
name_lst = []
df_lst = []
for path in path_lst:
    name = path.split("\\")[-1].split(".")[0]
    name_lst.append(name)
    df = gpd.read_file(path)
    df_province = df.groupby("Shengcode")[['C2022', 'C2025','C2030', 'C2035', 'C2040', 'C2050', 'C2060', 
        'E2022', 'E2025', 'E2030', 'E2035', 'E2040', 'E2050', 'E2060', 
        'P2022', 'P2025', 'P2030', 'P2035', 'P2040', 'P2050', 'P2060']].sum()
    df_province.reset_index(inplace=True)
    df_lst.append(df_province)
# Create an Excel writer object
with pd.ExcelWriter(r'C:\Users\ruome\Desktop\New folder\Renewable\ProcessingFiles\table\province_results_20250625.xlsx') as writer:
    # Write each DataFrame to a different sheet
    for i in range(len(name_lst)):
        sheet_name = f'{name_lst[i]}'
        df = df_lst[i]
        df.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
# summary one table 
path = r"C:\Users\ruome\Desktop\New folder\Renewable\论文编辑\数据输出\结果三-全生命周期折减\Solar_median_results_20250110gj.shp"
name = path.split("\\")[-1].split(".")[0]
shp_df = gpd.read_file(path)
# 需要统计的列
cols_to_count = ['KW2022', 'KW2025', 'KW2030', 'KW2035', 'KW2040', 'KW2050', 'KW2060']

# 计算各列 > 0 的数量
count_df = shp_df.groupby('Shengcode')[cols_to_count].apply(lambda x: (x > 0).sum()).reset_index()
count_df.columns = ['Shengcode'] + [f'{col}_count' for col in cols_to_count]

# 计算各列的总和
cols_to_sum = ['KW2022','KW2025', 'KW2030', 'KW2035', 'KW2040', 'KW2050', 'KW2060', 
       'C2022', 'C2025','C2030', 'C2035', 'C2040', 'C2050', 'C2060', 
       'E2022', 'E2025', 'E2030', 'E2035', 'E2040', 'E2050', 'E2060', 
       'P2022', 'P2025', 'P2030', 'P2035', 'P2040', 'P2050', 'P2060']
sum_df = shp_df.groupby('Shengcode')[cols_to_sum].sum().reset_index()
sum_df.columns = ['Shengcode'] + [f'{col}_sum' for col in cols_to_sum]

# 合并两个结果
summary_df = pd.merge(count_df, sum_df, on='Shengcode')
summary_df = pd.merge(df_shengcode, summary_df, on='Shengcode')

summary_df.to_excel(r"C:\Users\ruome\Desktop\New folder\Renewable\ProcessingFiles\table\敏感性统计表格\province_solar_median_kw2.xlsx")

In [ ]:
# summary one table 
path = r"C:\Users\ruome\Desktop\New folder\Renewable\论文编辑\数据输出\结果三-全生命周期折减\Solar_high_results_20250110gj.shp"
name = path.split("\\")[-1].split(".")[0]
shp_df = gpd.read_file(path)
# 需要统计的列
cols_to_count = ['KW2022', 'KW2025', 'KW2030', 'KW2035', 'KW2040', 'KW2050', 'KW2060']

# 计算各列 > 0 的数量
count_df = shp_df.groupby('Shengcode')[cols_to_count].apply(lambda x: (x > 0).sum()).reset_index()
count_df.columns = ['Shengcode'] + [f'{col}_count' for col in cols_to_count]

# 计算各列的总和
cols_to_sum = ['KW2022','KW2025', 'KW2030', 'KW2035', 'KW2040', 'KW2050', 'KW2060', 
       'C2022', 'C2025','C2030', 'C2035', 'C2040', 'C2050', 'C2060', 
       'E2022', 'E2025', 'E2030', 'E2035', 'E2040', 'E2050', 'E2060', 
       'P2022', 'P2025', 'P2030', 'P2035', 'P2040', 'P2050', 'P2060']
sum_df = shp_df.groupby('Shengcode')[cols_to_sum].sum().reset_index()
sum_df.columns = ['Shengcode'] + [f'{col}_sum' for col in cols_to_sum]

# 合并两个结果
summary_df = pd.merge(count_df, sum_df, on='Shengcode')
summary_df = pd.merge(df_shengcode, summary_df, on='Shengcode')

summary_df.to_excel(r"C:\Users\ruome\Desktop\New folder\Renewable\ProcessingFiles\table\敏感性统计表格\province_solar_high_kw2.xlsx")